# TTN遺伝子のイントロン同士の配列相同性をBLASTで評価する

**目的**
イントロン同士の配列を比較して、相同性を確認する<br>
相同性とスプライシングパターンの関連がないかを検証

In [1]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count
from lib.geneinfo import gene_id

import matplotlib.pyplot as plt
import pandas as pd

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

import subprocess
import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/home/du/Bcbio/mambaforge/envs/bio/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


## 準備

In [2]:
# 生物種の指定
specie = "human"

# ディレクトリのリセット
cmd1 = "rm -rf ../data/blast"
cmd2 = f"mkdir -p ../data/blast/db/{specie}/ ../data/blast/fasta/{specie}/ ../result/blast/{specie}/"
subprocess.call(cmd1.split())
subprocess.call(cmd2.split())

# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
# gbk.read_gbk(f"../data/gbk/{specie}_ttn.gb")
gbk.get_gbk('7273')

# 遺伝子全長配列
seq = gbk.gDNA_seq()

デフォルト値として、最もエクソンの多いNM_001267550.2を設定


## BLASTの設定

In [3]:
# すべてのイントロン配列をfasta形式で保存
with open(f"../data/blast/fasta/{specie}/ttn_intron.fasta","w") as f:
    for n,i in enumerate(gbk.intron_list()):
        f.write(f">intron{str(n+1).zfill(3)}\n")
        f.write("".join(seq[i[0]:i[1]]))
        f.write("\n")

# すべてのイントロン配列を個別にfasta形式で保存
for n,i in enumerate(gbk.intron_list()):
    with open(f"../data/blast/fasta/{specie}/ttn_intron{str(n+1).zfill(3)}.fasta","w") as f:
        f.write(f">intron{str(n+1).zfill(3)}\n")
        f.write("".join(seq[i[0]:i[1]]))

In [4]:
# BLASTデータベースの作成
cmd = f"makeblastdb -in ../data/blast/fasta/{specie}/ttn_intron.fasta -dbtype nucl -parse_seqids -out ../data/blast/db/{specie}/ttn_intron"
subprocess.run(cmd.split())



Building a new DB, current time: 11/18/2023 21:42:38
New DB name:   /home/du/GitHub/bio_chem/ttn_splicing/data/blast/db/human/ttn_intron
New DB title:  ../data/blast/fasta/human/ttn_intron.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 362 sequences in 0.143689 seconds.




CompletedProcess(args=['makeblastdb', '-in', '../data/blast/fasta/human/ttn_intron.fasta', '-dbtype', 'nucl', '-parse_seqids', '-out', '../data/blast/db/human/ttn_intron'], returncode=0)

## BLASTの実行

In [5]:
# すべてのイントロンに対してBLASTを実行
for i in range(gbk.intron_num()):
    fmt = 6
    cmd = f"blastn -query ../data/blast/fasta/{specie}/ttn_intron{str(i+1).zfill(3)}.fasta -db ../data/blast/db/{specie}/ttn_intron -out ../result/blast/{specie}/ttn_intron{str(i+1).zfill(3)}.txt -outfmt {fmt}"
    subprocess.run(cmd.split())

In [6]:
# BLAST結果のファイルリスト
ls = glob.glob(f"../result/blast/{specie}/ttn_intron*.txt")

# BLAST結果ファイルの読み込みと結合
df = pd.read_csv(ls[0], sep='\t', header=None)
for i in ls[1:]:
    tmp = pd.read_csv(i, sep='\t', header=None)
    df = pd.concat([df, tmp])
df.columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
df = df.sort_values('qseqid')
df

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,intron001,intron001,100.000,2556,0,0,1,2556,1,2556,0.000000e+00,4721.0
1,intron001,intron294,85.666,293,40,2,1848,2139,1551,1260,1.010000e-84,307.0
2,intron001,intron127,84.444,315,44,4,1838,2151,2002,1692,3.640000e-84,305.0
3,intron001,intron049,81.699,306,46,8,1839,2141,791,493,2.240000e-66,246.0
0,intron002,intron002,100.000,2210,0,0,1,2210,1,2210,0.000000e+00,4082.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,intron358,intron358,100.000,124,0,0,1,124,1,124,9.690000e-63,230.0
0,intron359,intron359,100.000,740,0,0,1,740,1,740,0.000000e+00,1367.0
0,intron360,intron360,100.000,100,0,0,1,100,1,100,1.650000e-49,185.0
0,intron361,intron361,100.000,525,0,0,1,525,1,525,0.000000e+00,970.0


In [7]:
df.to_csv('../data/blast/blast.csv')